# Working with AWS DMS tasks and Apache Hudi Deltastreamer 

Working with Apache Hudi Deltastreamer
HoodieDeltaStreamer utility is part of hudi-utilities-bundle that provides a way to ingest data from sources such as DFS or Kafka.

In this notebook, you will learn to use DeltaStreamer Utility to bulk insert data into a Hudi Dataset as a Copy on Write(CoW) and Merge on Write (MOR) using a DMS Full load and CDC task as a source.  

We will run queries in hudi-cli and SparkSQL to verify the tables and subsequent updates are incorporated into our datalake on Amazon S3

Let's get started !

## Generate DMS environment and Test Data

### Setup the DMS environment to generate Data

Generate a Test Database and a DMS Full Load and CDC tasks to S3, executing this 2 previous labs:

1) DMS Pre lab 1 (Create Test RDS Postgres) https://aws-dataengineering-day.workshop.aws/en/400/410-pre-lab-1.html

2) DMS Pre lab 2 (Create DMS environment) https://aws-dataengineering-day.workshop.aws/en/400/420-pre-lab-2.html

3) DMS Main Lab (Create and run Full Load Task) https://aws-dataengineering-day.workshop.aws/en/400/430-main-lab.html but instead on the step g. in the "Create Target endpoint section" https://aws-dataengineering-day.workshop.aws/en/400/430-main-lab.html#create-the-target-endpoint specify the following "Extra connection attributes" to generate Parquet data on S3:

addColumnName=true;dataFormat=parquet;enableStatistics=true;encodingType=rle-dictionary;maxFileSize=524,288;parquetTimestampInMillisecond=true;parquetVersion=PARQUET_2_0;rfc4180=false;timestampColumnName=op_cdc_timestamp;

<a href="https://ibb.co/wWyV3wd"><img src="https://i.ibb.co/gTyNHPr/dms-taregt-Params.png" alt="dms-taregt-Params" border="0"></a>

### Install Python AWS Wrangler and Psycopg lib

In [1]:
sc.install_pypi_package("psycopg2")
sc.install_pypi_package("awswrangler")


VBox()

VBox()

Starting Spark application
Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
11,application_1596580355664_0060,pyspark,idle,Link,Link,✔


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
11,application_1596580355664_0060,pyspark,idle,Link,Link,✔


...

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.
SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/a8/8f/1c5690eebf148d1d1554fc00ccf9101e134636553dbb75bdfef4f85d7647/psycopg2-2.8.5.tar.gz
    Complete output from command python setup.py egg_info:
    /usr/lib64/python3.7/distutils/dist.py:274: UserWarning: Unknown distribution option: 'project_urls'
      warnings.warn(msg)
    running egg_info
    creating pip-egg-info/psycopg2.egg-info
    writing pip-egg-info/psycopg2.egg-info/PKG-INFO
    writing dependency_links to pip-egg-info/psycopg2.egg-info/dependency_links.txt
    writing top-level names to pip-egg-info/psycopg2.egg-info/top_level.txt
    writing manifest file 'pip-egg-info/psycopg2.egg-info/SOURCES.txt'
    
    
    Error: pg_config executable not found.
    
    pg_config is required to build psycopg2 from source.  Please add the directory
    containing pg_config to the $PATH or specify the full executable path with the
    option:
    
        python setup.py build_ext --pg-config /path/to/pg_config build ...
   

## Copy Hudi Libraries on your EMR (5.30) cluster 

0. For the following steps to work, you should have launched the EMR cluster with appropriate permissions set for **Systems Manager Session Manager** 
1. From the AWS Console, type SSM in the search box and navigate to the **Amazon System Manager console**
2. On the left hand side, select **Session Manager** from **Instances and Nodes** section
3. Click on the start session and you should see two EC2 instances listed 
4. Select instance-id of the **EMR's Master** Node and click on **Start session**
5. From the terminal type the following to change to user *ec2-user*
 
```bash
sh-4.2$ sudo su hadoop
hadoop@ip-10-0-2-73 /]$ cd
[hadoop@ip-10-0-2-73 ~]$ hdfs dfs -mkdir -p /apps/hudi/lib
[hadoop@ip-10-0-2-73 ~]$ hdfs dfs -copyFromLocal /usr/lib/hudi/hudi-spark-bundle.jar /apps/hudi/lib/hudi-spark-bundle.jar
hadoop@ip-10-0-2-73 ~]$ hdfs dfs -copyFromLocal /usr/lib/spark/external/lib/spark-avro.jar /apps/hudi/lib/spark-avro.jar
hadoop@ip-10-0-2-73 ~]$ hdfs dfs -copyFromLocal /usr/lib/hudi/hudi-utilities-bundle.jar /apps/hudi/lib/hudi-utilities-bundle.jar
hadoop@ip-10-0-2-73 ~]$ hdfs dfs -copyFromLocal /usr/lib/spark/jars/httpclient-4.5.9.jar /apps/hudi/lib/httpclient-4.5.9.jar
[hadoop@ip-10-0-2-73 ~]$ hdfs dfs -ls /apps/hudi/lib/
Found 4 items
-rw-r--r--   1 hadoop hadoop     774384 2020-05-06 05:11 /apps/hudi/lib/httpclient-4.5.9.jar
-rw-r--r--   1 hadoop hadoop   20967361 2020-05-06 05:10 /apps/hudi/lib/hudi-spark-bundle.jar
-rw-r--r--   1 hadoop hadoop   39051878 2020-05-06 05:10 /apps/hudi/lib/hudi-utilities-bundle.jar
-rw-r--r--   1 hadoop hadoop     187458 2020-05-06 05:10 /apps/hudi/lib/spark-avro.jar
```



## Run DeltaStreamer to write a Copy on Write (COW) table

We will now run the DeltaStreamer utility as an EMR Step to write the above JSON formatted data into a Hudi dataset. To do that, we will need the following:

* Properties file on localfs or dfs, with configurations for Hudi client, schema provider, key generator and data source 
* Schema file for source dataset
* Schema file for target dataset



To run DeltStreamer Replace the following values in the below command in the text editor

1. Set the cluster-id with the value from your EMR 5.3.0 cluster
2. Replace xxxx part  with the S3 bucket name 
3. For -- target-base-path value with the S3 bucket name
4. After replacing the values, execute the cell
5. If the values are replaced correctly, you should see a step id displayed as the output



In [ ]:
import awswrangler as wr
import boto3

boto3.setup_default_session(region_name="us-west-2")

## Submit the bulk insert job to create COW table

In [3]:
#COW
# SOURCE TABLE sporting_event_ticket

cluster_id = "j-XXXXXXX"

command= f"spark-submit --class org.apache.hudi.utilities.deltastreamer.HoodieDeltaStreamer  \
--jars /usr/lib/spark/external/lib/spark-avro_2.11-2.4.5-amzn-0.jar  \
--master yarn  \
file:///usr/lib/hudi/hudi-utilities-bundle_2.11-0.5.2-incubating.jar  \
--table-type COPY_ON_WRITE  \
--source-class org.apache.hudi.utilities.sources.ParquetDFSSource  \
--source-ordering-field op_cdc_timestamp  \
--target-base-path s3://XXXXX/landing-hudi/co/tickets/dms_sample/hudi_sporting_event_ticket  \
--target-table hudi_sporting_event_ticket  \
--transformer-class org.apache.hudi.utilities.transform.AWSDmsTransformer  \
--payload-class org.apache.hudi.payload.AWSDmsAvroPayload  \
--enable-hive-sync  \
--hoodie-conf hoodie.deltastreamer.source.dfs.root=s3://XXXXXX/landing-hudi/co/tickets/dms_sample/sporting_event_ticket   \
--hoodie-conf hoodie.datasource.write.recordkey.field=id  \
--hoodie-conf hoodie.datasource.write.partitionpath.field=seat_row \
--hoodie-conf hoodie.datasource.hive_sync.partition_fields=seat_row \
--hoodie-conf hoodie.datasource.hive_sync.database=default  \
--hoodie-conf hoodie.datasource.hive_sync.table=hudi_sporting_event_ticket   \
--hoodie-conf hoodie.datasource.hive_sync.partition_extractor_class=org.apache.hudi.hive.MultiPartKeysValueExtractor  \
"

#Submit the job to the specified cluster (step)
step_id = wr.emr.submit_step(cluster_id, command)
print(step_id)

VBox()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s-H8F2Q4SO16UCs-H8F2Q4SO16UC

## Submit job to create Merge on Read (MOR) table -- (Optional)

In [10]:
#MOR
# TABLE sporting_event

cluster_id = "j-2FBL7NHLI23IQ"

command= f"spark-submit --class org.apache.hudi.utilities.deltastreamer.HoodieDeltaStreamer  \
--jars /usr/lib/spark/external/lib/spark-avro_2.11-2.4.5-amzn-0.jar  \
--master yarn  \
file:///usr/lib/hudi/hudi-utilities-bundle_2.11-0.5.2-incubating.jar  \
--table-type MERGE_ON_READ  \
--op BULK_INSERT \
--source-class org.apache.hudi.utilities.sources.ParquetDFSSource  \
--source-ordering-field op_cdc_timestamp  \
--target-base-path s3://XXXXXX/landing-hudi/co/tickets/dms_sample/hudi_sporting_event_ticket_mor  \
--target-table hudi_sporting_event_ticket_mor  \
--transformer-class org.apache.hudi.utilities.transform.AWSDmsTransformer  \
--payload-class org.apache.hudi.payload.AWSDmsAvroPayload  \
--enable-hive-sync  \
--hoodie-conf hoodie.deltastreamer.source.dfs.root=s3://XXXXXXX/landing-hudi/co/tickets/dms_sample/sporting_event_ticket   \
--hoodie-conf hoodie.datasource.write.recordkey.field=id  \
--hoodie-conf hoodie.datasource.write.partitionpath.field=seat_row \
--hoodie-conf hoodie.datasource.hive_sync.partition_fields=seat_row \
--hoodie-conf hoodie.datasource.hive_sync.database=default  \
--hoodie-conf hoodie.datasource.hive_sync.table=hudi_sporting_event_ticket_mor   \
--hoodie-conf hoodie.datasource.hive_sync.partition_extractor_class=org.apache.hudi.hive.MultiPartKeysValueExtractor"


#Submit the job to the specified cluster (step)
step_id = wr.emr.submit_step(cluster_id, command)
print(step_id)

VBox()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s-2U2MAIP88WEMs-2U2MAIP88WEM

Now let us check the S3 path:

```
$ aws s3 ls s3://<my bucket>/landing-hudi/co/tickets/dms_sample/hudi_sporting_event_ticket/
                           PRE .hoodie/
2020-05-06 05:29:08          0 .hoodie_$folder$
2020-05-06 05:29:28         93 .hoodie_partition_metadata
2020-05-06 05:29:32    2317411 19cb0a3f-01f5-4590-a9de-df4edeb32125-0_4-4-36_20200506052917.parquet
2020-05-06 05:29:30    2080361 2903b865-e188-4079-8e1a-3afeefe6543b-0_5-4-37_20200506052917.parquet
2020-05-06 05:29:32    2244907 3c454bf3-eb20-41d4-941b-cec82c10db6c-0_9-4-41_20200506052917.parquet
2020-05-06 05:29:31    2366016 440f2437-e94f-4580-88a8-7179b40e4400-0_2-4-34_20200506052917.parquet
2020-05-06 05:29:32    2305575 45b8bd53-7285-44cf-88c5-40aedb70e17d-0_0-4-32_20200506052917.parquet
2020-05-06 05:29:31    2234887 7cca038b-8e27-4ddd-8b30-521c85b4ba07-0_1-4-33_20200506052917.parquet
2020-05-06 05:29:32    2205694 9ede479c-3d7c-4b63-ba43-c9ba26ddb5bb-0_7-4-39_20200506052917.parquet
2020-05-06 05:29:32    2559101 b9af4f18-7663-4e73-ae3f-4b0703726f80-0_6-4-38_20200506052917.parquet
2020-05-06 05:29:32    2094234 c1bc770d-358c-48c5-8f47-f5b5a94e3867-0_8-4-40_20200506052917.parquet
2020-05-06 05:29:31    2246656 f671a969-6a6f-4759-a12d-c768dcac49f2-0_3-4-35_20200506052917.parquet
```

To query the Hudi dataset you can do one of the following

1. - GO to Athena on AWS Console and look for 'default' database and the table'hudi_sport_tickets_events' 
2. - Navigate to the another sparkmagic notebook and run queries in Spark using SparkMagic cell
- SSH to the master node (you can also SSM if you launched your cluster with SSM permissions) and run queries using Hive/Presto
- Head to the Hue console on Amazon EMR and run queries

Let us use SparkSQL for instance. Login to your EMR cluster and start SparkSQL. 

```

$ spark-sql --jars hdfs:///apps/lib/hudi/*.jar

spark-sql> select * from profile_cow limit 2;

20200506052917	20200506052917_6_1	993739fd-97af-494e-8920-267b4796a686		b9af4f18-7663-4e73-ae3f-4b0703726f80-0_6-4-38_20200506052917.parquet	Stacy Lester	142-413-1721x450	Advertising art director	Harris Group	426-69-1109	USS May
FPO AE 08308	1921-08-21	coxrachel@gmail.com	2011-11-07T06:10:11
20200506052917	20200506052917_6_2	993776e5-d124-4c12-8835-e322690fcfea		b9af4f18-7663-4e73-ae3f-4b0703726f80-0_6-4-38_20200506052917.parquet	Kayla Boone	(705)940-6720x57967	Physiological scientist	Salazar-Alexander	882-54-5400	4702 Steven Ports
Tracystad, CT 97143	1972-05-18	kristie74@ward.com	2017-01-05T11:50:44
Time taken: 9.263 seconds, Fetched 2 row(s)

Now, lets make a note of street_address in one of these two records.

spark-sql> select _hoodie_commit_time, street_address from profile_cow where _hoodie_record_key="993739fd-97af-494e-8920-267b4796a686";

20200506052917	USS May FPO AE 08308     

Time taken: 3.86 seconds, Fetched 1 row(s)

```

Lets now run an upsert to observe the change in records


## Run some updates on the source Postgres database

You can run updates now on the fake profile data generated. 

In [ ]:
#Connect an make an Update query on the source RDS Postgres
import psycopg2


connection = psycopg2.connect(user = "master",
                              password = "master123",
                              host = "XXXXXXXXXXXX.us-west-2.rds.amazonaws.com",
                              port = "5432",
                              database = "sportstickets")

cursor = connection.cursor()
# Print PostgreSQL Connection properties
print ( connection.get_dsn_parameters(),"\n")

#Make some updates to the price to some existing records on the source RDS Postgres sporting_event_ticket table

postgreSQL_select_Query = "BEGIN;   \
UPDATE dms_sample.sporting_event_ticket SET ticket_price = '111.99' WHERE  id = '1';  \
UPDATE dms_sample.sporting_event_ticket SET ticket_price = '111.99' WHERE  id = '11'; \
UPDATE dms_sample.sporting_event_ticket SET ticket_price = '111.99' WHERE  id = '21'; \
UPDATE dms_sample.sporting_event_ticket SET ticket_price = '111.99' WHERE  id = '31'; \
COMMIT;"

cursor.execute(postgreSQL_select_Query)
print("Executing update query")



## Start DeltaStreamer upsert job on COW Table to update the records

In [4]:
cluster_id = "j-XXXXXXX"

# SOURCE TABLE sporting_event_ticket 

command= f"spark-submit --class org.apache.hudi.utilities.deltastreamer.HoodieDeltaStreamer  \
--jars /usr/lib/spark/external/lib/spark-avro_2.11-2.4.5-amzn-0.jar  \
--master yarn  \
file:///usr/lib/hudi/hudi-utilities-bundle_2.11-0.5.2-incubating.jar  \
--table-type COPY_ON_WRITE  \
--payload-class org.apache.hudi.payload.AWSDmsAvroPayload  \
--op UPSERT \
--source-class org.apache.hudi.utilities.sources.ParquetDFSSource  \
--source-ordering-field op_cdc_timestamp  \
--target-base-path s3://XXXXXX/landing-hudi/co/tickets/dms_sample/hudi_sporting_event_ticket  \
--target-table hudi_sporting_event_ticket  \
--transformer-class org.apache.hudi.utilities.transform.AWSDmsTransformer  \
--enable-hive-sync  \
--hoodie-conf hoodie.deltastreamer.source.dfs.root=s3://XXXXXX/landing-hudi/co/tickets/dms_sample/sporting_event_ticket   \
--hoodie-conf hoodie.datasource.write.recordkey.field=id  \
--hoodie-conf hoodie.datasource.write.partitionpath.field=seat_row \
--hoodie-conf hoodie.datasource.hive_sync.partition_fields=seat_row \
--hoodie-conf hoodie.datasource.hive_sync.database=default  \
--hoodie-conf hoodie.datasource.hive_sync.table=hudi_sporting_event_ticket   \
--hoodie-conf hoodie.datasource.hive_sync.partition_extractor_class=org.apache.hudi.hive.MultiPartKeysValueExtractor"

#Submit the job to the specified cluster (step)
step_id = wr.emr.submit_step(cluster_id, command)
print(step_id)

VBox()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s-SDJRP8DOGS7Ys-SDJRP8DOGS7Y

Check the records in updates/ location.

```
$ aws s3 ls s3://XXXXXXX-hudi-dms-cdc/landing-hudi/co/tickets/dms_sample/hudi_sporting_event_ticket
2020-05-06 05:47:53    3686902 profile_1.json
2020-05-06 05:48:40    3686509 profile_10.json
2020-05-06 05:48:46    3682899 profile_11.json
2020-05-06 05:48:51    3685701 profile_12.json
2020-05-06 05:48:56    3682880 profile_13.json
2020-05-06 05:49:02    3687458 profile_14.json
2020-05-06 05:47:58    3686045 profile_2.json
2020-05-06 05:48:03    3686088 profile_3.json
2020-05-06 05:48:08    3687066 profile_4.json
2020-05-06 05:48:14    3686211 profile_5.json
2020-05-06 05:48:19    3685343 profile_6.json
2020-05-06 05:48:24    3687729 profile_7.json
2020-05-06 05:48:30    3685578 profile_8.json
2020-05-06 05:48:35    3686606 profile_9.json

```

## Query the updated Hudi Dataset

Now lets check the S3 path of output location. Notice the new Parquet files. 

```

$ aws s3 ls s3://558139421422-hudi-dms-cdc/landing-hudi/co/tickets/dms_sample/hudi_sporting_event_ticket
                           PRE .hoodie/
2020-05-06 05:29:08          0 .hoodie_$folder$
2020-05-06 05:29:28         93 .hoodie_partition_metadata
2020-05-06 05:29:32    2317411 19cb0a3f-01f5-4590-a9de-df4edeb32125-0_4-4-36_20200506052917.parquet
2020-05-06 06:32:48    2320387 19cb0a3f-01f5-4590-a9de-df4edeb32125-0_8-21-135_20200506063222.parquet
2020-05-06 05:29:30    2080361 2903b865-e188-4079-8e1a-3afeefe6543b-0_5-4-37_20200506052917.parquet
2020-05-06 06:32:49    2085116 2903b865-e188-4079-8e1a-3afeefe6543b-0_7-21-134_20200506063222.parquet
2020-05-06 06:32:47    2248999 3c454bf3-eb20-41d4-941b-cec82c10db6c-0_5-21-132_20200506063222.parquet
2020-05-06 05:29:32    2244907 3c454bf3-eb20-41d4-941b-cec82c10db6c-0_9-4-41_20200506052917.parquet
2020-05-06 05:29:31    2366016 440f2437-e94f-4580-88a8-7179b40e4400-0_2-4-34_20200506052917.parquet
2020-05-06 06:32:48    2369677 440f2437-e94f-4580-88a8-7179b40e4400-0_9-21-136_20200506063222.parquet
2020-05-06 05:29:32    2305575 45b8bd53-7285-44cf-88c5-40aedb70e17d-0_0-4-32_20200506052917.parquet
2020-05-06 06:32:49    2310293 45b8bd53-7285-44cf-88c5-40aedb70e17d-0_4-21-131_20200506063222.parquet
2020-05-06 05:29:31    2234887 7cca038b-8e27-4ddd-8b30-521c85b4ba07-0_1-4-33_20200506052917.parquet
2020-05-06 06:32:49    2238812 7cca038b-8e27-4ddd-8b30-521c85b4ba07-0_6-21-133_20200506063222.parquet
2020-05-06 06:32:48    2209143 9ede479c-3d7c-4b63-ba43-c9ba26ddb5bb-0_2-21-129_20200506063222.parquet
2020-05-06 05:29:32    2205694 9ede479c-3d7c-4b63-ba43-c9ba26ddb5bb-0_7-4-39_20200506052917.parquet
2020-05-06 06:32:49    2563854 b9af4f18-7663-4e73-ae3f-4b0703726f80-0_1-21-128_20200506063222.parquet
2020-05-06 05:29:32    2559101 b9af4f18-7663-4e73-ae3f-4b0703726f80-0_6-4-38_20200506052917.parquet
2020-05-06 06:32:48    2098070 c1bc770d-358c-48c5-8f47-f5b5a94e3867-0_3-21-130_20200506063222.parquet
2020-05-06 05:29:32    2094234 c1bc770d-358c-48c5-8f47-f5b5a94e3867-0_8-4-40_20200506052917.parquet
2020-05-06 06:32:47    2249129 f671a969-6a6f-4759-a12d-c768dcac49f2-0_0-21-127_20200506063222.parquet
2020-05-06 05:29:31    2246656 f671a969-6a6f-4759-a12d-c768dcac49f2-0_3-4-35_20200506052917.parquet

```

Let's query an upserted record. 

```
$ spark-sql --jars hdfs:///apps/lib/hudi/*.jar 

spark-sql> select _hoodie_commit_time, street_address from profile_cow where _hoodie_record_key="993739fd-97af-494e-8920-267b4796a686";

20200506052917	USS May FPO AE 08308                                     # Old address 
20200506063222	880 Jackson Mountains Suite 221 Lake Kristin, SC 39264   # Our recent update 

Time taken: 7.901 seconds, Fetched 2 row(s)

```

Now lets check out Hudi CLI

```

hudi:person_profile_cow->commits show
20/05/06 06:47:14 INFO timeline.HoodieActiveTimeline: Loaded instants java.util.stream.ReferencePipeline$Head@d689f6
20/05/06 06:47:15 INFO s3n.S3NativeFileSystem: Opening 's3://XXXXXXX/landing-hudi/co/tickets/dms_sample/sporting_event_ticket/.hoodie/20200506063222.commit' for reading
20/05/06 06:47:15 INFO s3n.S3NativeFileSystem: Opening 's3://XXXXX/landing-hudi/co/tickets/dms_sample/sporting_event_ticket/.hoodie/20200506052917.commit' for reading
╔════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime     │ Total Bytes Written │ Total Files Added │ Total Files Updated │ Total Partitions Written │ Total Records Written │ Total Update Records Written │ Total Errors ║
╠════════════════╪═════════════════════╪═══════════════════╪═════════════════════╪══════════════════════════╪═══════════════════════╪══════════════════════════════╪══════════════╣
║ 20200506063222 │ 21.6 MB             │ 0                 │ 10                  │ 1                        │ 150000                │ 140000                       │ 0            ║
╟────────────────┼─────────────────────┼───────────────────┼─────────────────────┼──────────────────────────┼───────────────────────┼──────────────────────────────┼──────────────╢
║ 20200506052917 │ 21.6 MB             │ 10                │ 0                   │ 1                        │ 150000                │ 0                            │ 0            ║
╚════════════════╧═════════════════════╧═══════════════════╧═════════════════════╧══════════════════════════╧═══════════════════════╧══════════════════════════════╧══════════════╝

```